In [ ]:
import codecs
import json 
import string
import os
import nltk
import re
import ast 
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
f = open('/content/drive/MyDrive/IRE/postinglist.json')             #loading posting list
postings = json.load(f)

In [ ]:
f1 = open('/content/drive/MyDrive/IRE/invertedIndexing.json')              #loading the inverted indexing
inverted_indexing = json.load(f1)

In [ ]:
f2 = open('/content/drive/MyDrive/IRE/title_indexing.json')                         #loading title indexing
title_indexing = json.load(f2)

In [ ]:
def preprocessing(texts):                                      #preprocessing function
    #word_tokenize
#     print(texts)
#     print()
    letters_only = re.sub("[^a-zA-Z]",  # Search for all non-letters
                          " ",          # Replace all non-letters with spaces
                          texts)
#     print(letters_only)
#     print()
    word_tokens = word_tokenize(letters_only)
#     print(word_tokens)
#     print()
    #Lemmatization
    lemmatizer = WordNetLemmatizer()
    word_lemm = [lemmatizer.lemmatize(word) for word in word_tokens if word not in stopwords.words('english')]
#     print(word_lemm)
#     print()
    return word_lemm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
title = title_indexing.keys()                 #showing how many documents are present
title = list(title)
len(title)

467

In [ ]:
def OR_merge(val,val1):                         #Merge or function for OR operation.
  i,j =0,0 
  compare = 0
  doc = []
  while i<len(val) and j<len(val1):
    
    compare = compare + 1
    if val[i] == val1[j]:
      doc.append(val[i])
    else:
      doc.append(val[i])
      doc.append(val[j])
    i = i+1
    j = j+1   
  if i<len(val):
    for x in range(i,len(val)):
      doc.append(val[x])
  elif j<len(val1):
    for x in range(j,len(val1)):
      doc.append(val1[x])

  return compare,doc

In [ ]:
def AND_merge(val,val1):                      #merge and function
  i,j = 0,0 
  compare = 0
  doc =[]
  while i < len(val) and j < len(val1):
    compare = compare + 1
    if val[i] == val1[j]:
      doc.append(val[i])
      i = i+1
      j = j+1
    elif val[i] < val1[j]:
      i = i + 1
    elif val[i] > val1[j]:
      j = j+1    

  return compare,doc    

In [ ]:
def AND_NOT(val,val1):                      #merge and function
  i,j = 0,0 
  compare = 0
  doc =[]
  while i < len(val) and j < len(val1):
    compare = compare + 1
    if val[i] == val1[j]:
      i = i+1
      j = j+1
    elif val[i] < val1[j]:
      doc.append(val[i])
      i = i + 1
    elif val[i] > val1[j]:
      j = j+1    

  return compare,doc    

In [ ]:
def OR_NOT(val,val1):
  i,j = 0,0 
  compare = 0
  doc =[]
  while i < len(val) and j < len(val1):
    compare = compare + 1
    if val[i] == val1[j]:
      doc.append(val[i])
      i = i+1
      j = j+1
    elif val[i] < val1[j]:
      doc.append(val[i])
      i = i + 1
    elif val[i] > val1[j]:
      j = j+1    

  return compare,doc 

In [ ]:
input1 = input("Enter the Query")                 # input query
print("Input Query is :",input1)
query = preprocessing(input1.lower())          #preprocessing
print("Tokens are:",query)                      #getting tokens after preprocessing
i = 0
j = 0
res = []
compare1 = 0
while i < len(query):         
  print("Enter 1 for OR operation")
  print("Enter 1 for And operation")
  print("Enter 3 for OR NOT operation")
  print("Enter 4 for And NOT operation")
  choice = input("Enter the desired Operation")
  choice = int(choice)
  
  if j == 0:
    j = 1
    i = i+2
    term1 = query[0]
    term2 = query[1]
    print(term1)
    
    
    k = inverted_indexing[term1]
    k = str(k)
    
    val = postings[k].keys()
    val = list(val)

    k1 = inverted_indexing[term2]
    k1 = str(k1)
    val1 = postings[k1].keys()
    val1 = list(val1)
    print("val size",len(val))
    print("val1 size",len(val1))


    #print("true")
    if choice == 1:                 #for OR operation
       #print("true")
       
       
       final_res = list(set(val1).union(set(val)))
       #res.append(final_res)
       print("result length",len(final_res))
       compare,doc = OR_merge(val,val1)
       compare1 = compare1 + compare

    if choice == 2:             #for AND operation
       #print("true")
       if len(val) >= len(val1):
         final_res = list(set(val1).intersection(set(val)))

       else:
           final_res = list(set(val).intersection(set(val1)))

       compare,doc = AND_merge(val,val1)
       compare1= compare + compare1    
       
       print("result length",len(final_res))   

    if choice == 3:                   #for OR NOT operation
      without_val1 = set(title).difference(set(val1))
      final_res = list(set(val).union(set(without_val1)))
      compare,doc = OR_NOT(val,val1)
      compare1= compare + compare1    
      print("result length",len(final_res))

    if choice == 4:                #for AND NOT operation
      without_val1 = set(title).difference(set(val1))
      if len(without_val1) > len(val):
          final_res = list(set(val).intersection(set(without_val1)))


      else :
         final_res = list(set(without_val1).intersection(set(val)))

      compare,doc = AND_NOT(val,val1)
      compare1= compare + compare1       


      print("result length",len(final_res))  

  else:       #query for other than first two tokens
    
    j = 1
    term = query[i]
    k = inverted_indexing[term]
    k = str(k)
    val = postings[k].keys()
    val = list(val)
    print("val size",len(val))
    print("final_size size",len(final_res))
    if choice == 1:
       
       final_res = list(set(final_res).union(set(val)))
       print("result length",len(final_res))
       compare,doc =  OR_merge(doc,val)
       compare1 = compare + compare1


    if choice == 2:
       #print("true")
       if len(val) >= len(final_res):
         final_res = list(set(final_res).intersection(set(val)))

       else:
           final_res = list(set(val).intersection(set(final_res)))

       compare,doc = AND_merge(doc,val)
       compare1 = compare + compare1     
       
       print("result length",len(final_res))   

    if choice == 3:
         without_val = set(title).difference(set(val))
         final_res = list(set(final_res).union(set(without_val)))
         print("result length",len(final_res))
         compare,doc = OR_NOT(val,val1)
         compare1= compare + compare1    

    if choice == 4:
      without_val = set(title).difference(set(val))
      if len(without_val) > len(val):
         final_res = list(set(final_res).intersection(set(without_val)))

      else:
        final_res = list(set(without_val).intersection(set(final_res)))

      compare,doc = AND_NOT(val,val1)
      compare1= compare + compare1      

      print("result length",len(final_res))  

    i = i+1   

print("lenth of result",len(final_res))
print(final_res)
print("no of comparisions",compare1)
#print("lenth of doc",len(doc))

         

Enter the Querytelephone paved roads.
Input Query is : telephone paved roads.
Tokens are: ['telephone', 'paved', 'road']
Enter 1 for OR operation
Enter 1 for And operation
Enter 3 for OR NOT operation
Enter 4 for And NOT operation
Enter the desired Operation3
telephone
val size 36
val1 size 8
result length 463
Enter 1 for OR operation
Enter 1 for And operation
Enter 3 for OR NOT operation
Enter 4 for And NOT operation
Enter the desired Operation4
val size 121
final_size size 463
result length 345
lenth of result 345
['sre09.txt', 'gold3ber.txt', 'wanderer.fun', 'jackbstl.txt', 'lrrhood.txt', 'bulnland.txt', 'greatlrn.leg', 'grav', 'blasters.fic', 'horsdonk.txt', 'gemdra.txt', 'contrad1.hum', 'ltp', 'hotline1.txt', 'mindwar', 'quarter.c2', 'quarter.c11', 'blossom.pom', 'sre07.txt', 'buldream.txt', 'cow.exploder', 'bishop00.txt', 'mario.txt', 'idi.hum', 'elveshoe.txt', 'hole2nar.txt', 'lure.txt', 'ghost', 'sre02.txt', 'poem-1.txt', 'bulolli1.txt', 'blackrdr', 'korea.s', 'weeprncs.txt', '

In [ ]:
Lion stood thoughtfully for a moment.